# HW \#2

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
import gensim
import numpy as np
import math
import nltk
from nltk.stem.snowball import SnowballStemmer 
from nltk.corpus import stopwords
import spacy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from gensim.models import FastText
import warnings
warnings.filterwarnings('ignore')

## Parse xml

In [10]:
class XML2DataFrame:
    def __init__(self, xml_path):
        xml_data = open(xml_path, encoding='utf-8')
        self.root = ET.XML(xml_data.read())[1]

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.attrib["name"]] = None if element.text == "NULL" else element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [11]:
tkk_train_raw = XML2DataFrame("SentiRuEval/tkk_train_2016.xml").process_data().fillna(0)
bank_train_raw = XML2DataFrame("SentiRuEval/bank_train_2016.xml").process_data().fillna(0)

tkk_test_raw = XML2DataFrame("SentiRuEval/tkk_test_etalon.xml").process_data().fillna(0)
bank_test_raw = XML2DataFrame("SentiRuEval/banks_test_etalon.xml").process_data().fillna(0)

## Construct train and test

In [12]:
tkk_train_text = tkk_train_raw['text'].values
tkk_test_text = tkk_test_raw['text'].values
tkk_train_labels = tkk_train_raw[["beeline", "komstar", "mts", "rostelecom", "skylink", "tele2"]].astype(int).sum(axis=1).values
tkk_test_labels = tkk_test_raw[["beeline", "komstar", "mts", "rostelecom", "skylink", "tele2"]].astype(int).sum(axis=1).values

bank_train_text = bank_train_raw['text'].values
bank_test_text = bank_test_raw['text'].values
bank_train_labels = bank_train_raw[['alfabank','bankmoskvy','gazprom','raiffeisen','rshb','sberbank','uralsib','vtb']].astype(int).sum(axis=1).values
bank_test_labels = bank_test_raw[['alfabank','bankmoskvy','gazprom','raiffeisen','rshb','sberbank','uralsib','vtb']].astype(int).sum(axis=1).values

## Normalize labels

In [25]:
# ~= np.array([math.copysign(x/x, x) if x !=0 else 0 for x in s]) 
def normalize_number(x):
    return int(math.copysign(x/x, x)) if x !=0 else 0
def normalize_series(s):
    return np.array([normalize_number(x) for x in s])

In [14]:
tkk_train_labels = normalize_series(tkk_train_labels)
tkk_test_labels = normalize_series(tkk_test_labels)

bank_train_labels = normalize_series(bank_train_labels)
bank_test_labels = normalize_series(bank_test_labels)

## Preprocess text: tokenize, stem, delete stopwords

In [17]:
stop_words = stopwords.words('russian')
sb_stemmer = SnowballStemmer('russian')

def tokenize(text):
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    tokens = [sb_stemmer.stem(token) for token in tokens if token not in stop_words]
    text = " ".join(tokens)
    
    return text

In [18]:
tkk_train_text = [tokenize(text) for text in tkk_train_text]
tkk_test_text = [tokenize(text) for text in tkk_test_text]

bank_train_text = [tokenize(text) for text in bank_train_text]
bank_test_text = [tokenize(text) for text in bank_test_text]

# CNNs

## TKK

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

### Data prepration

In [22]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(tkk_train_text)

tkk_X_train = pad_sequences(tokenizer.texts_to_sequences(tkk_train_text), maxlen=100)
tkk_X_test = pad_sequences(tokenizer.texts_to_sequences(tkk_test_text), maxlen=100)

word_index = tokenizer.word_index

In [35]:
tkk_X_train.shape

(8643, 100)

In [27]:
encoder = LabelBinarizer()
encoder.fit(tkk_train_labels)
tkk_y_train = encoder.transform(tkk_train_labels)
tkk_y_test = encoder.transform(tkk_test_labels)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [29]:
tkk_y_train.shape

(8643, 3)

In [33]:
tkk_y_train[:5]

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0]])

### Train model

In [59]:
model = Sequential()

model.add(Embedding(10000, 1000, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(tkk_X_train, tkk_y_train, batch_size=32, epochs=5)

Epoch 1/5


6592/8643 [=====================>........] - ETA: 10:05 - loss: 0.6939 - acc: 0.54 - ETA: 6:16 - loss: 0.6594 - acc: 0.6146 - ETA: 4:58 - loss: 0.6353 - acc: 0.638 - ETA: 4:21 - loss: 0.6181 - acc: 0.666 - ETA: 3:56 - loss: 0.6214 - acc: 0.662 - ETA: 3:40 - loss: 0.6064 - acc: 0.680 - ETA: 3:29 - loss: 0.5834 - acc: 0.702 - ETA: 3:21 - loss: 0.5980 - acc: 0.690 - ETA: 3:13 - loss: 0.5982 - acc: 0.692 - ETA: 3:08 - loss: 0.5961 - acc: 0.693 - ETA: 3:04 - loss: 0.5838 - acc: 0.706 - ETA: 3:01 - loss: 0.5785 - acc: 0.710 - ETA: 2:57 - loss: 0.5805 - acc: 0.706 - ETA: 2:54 - loss: 0.5759 - acc: 0.712 - ETA: 2:52 - loss: 0.5681 - acc: 0.720 - ETA: 2:49 - loss: 0.5653 - acc: 0.722 - ETA: 2:47 - loss: 0.5680 - acc: 0.720 - ETA: 2:45 - loss: 0.5637 - acc: 0.724 - ETA: 2:43 - loss: 0.5586 - acc: 0.729 - ETA: 2:42 - loss: 0.5558 - acc: 0.731 - ETA: 2:40 - loss: 0.5566 - acc: 0.730 - ETA: 2:39 - loss: 0.5551 - acc: 0.731 - ETA: 2:37 - loss: 0.5539 - acc: 0.731 - ETA: 2:36 - loss: 0.5518 - acc: 0.

8643/8643 [==============================] - ETA: 36s - loss: 0.4132 - acc: 0.81 - ETA: 36s - loss: 0.4132 - acc: 0.81 - ETA: 35s - loss: 0.4125 - acc: 0.81 - ETA: 34s - loss: 0.4119 - acc: 0.81 - ETA: 34s - loss: 0.4119 - acc: 0.81 - ETA: 33s - loss: 0.4109 - acc: 0.81 - ETA: 33s - loss: 0.4107 - acc: 0.81 - ETA: 32s - loss: 0.4103 - acc: 0.81 - ETA: 32s - loss: 0.4104 - acc: 0.81 - ETA: 31s - loss: 0.4099 - acc: 0.81 - ETA: 30s - loss: 0.4098 - acc: 0.81 - ETA: 30s - loss: 0.4091 - acc: 0.81 - ETA: 29s - loss: 0.4089 - acc: 0.81 - ETA: 29s - loss: 0.4086 - acc: 0.81 - ETA: 28s - loss: 0.4085 - acc: 0.81 - ETA: 27s - loss: 0.4080 - acc: 0.81 - ETA: 27s - loss: 0.4081 - acc: 0.81 - ETA: 26s - loss: 0.4075 - acc: 0.81 - ETA: 26s - loss: 0.4081 - acc: 0.81 - ETA: 25s - loss: 0.4076 - acc: 0.81 - ETA: 25s - loss: 0.4067 - acc: 0.81 - ETA: 24s - loss: 0.4063 - acc: 0.81 - ETA: 23s - loss: 0.4064 - acc: 0.81 - ETA: 23s - loss: 0.4062 - acc: 0.81 - ETA: 22s - loss: 0.4059 - acc: 0.81 - ETA: 

6592/8643 [=====================>........] - ETA: 2:30 - loss: 0.2169 - acc: 0.916 - ETA: 2:31 - loss: 0.2114 - acc: 0.916 - ETA: 2:31 - loss: 0.1909 - acc: 0.937 - ETA: 2:30 - loss: 0.1950 - acc: 0.932 - ETA: 2:31 - loss: 0.1907 - acc: 0.935 - ETA: 2:30 - loss: 0.1916 - acc: 0.935 - ETA: 2:29 - loss: 0.1967 - acc: 0.927 - ETA: 2:28 - loss: 0.2166 - acc: 0.916 - ETA: 2:28 - loss: 0.2004 - acc: 0.925 - ETA: 2:28 - loss: 0.1934 - acc: 0.929 - ETA: 2:27 - loss: 0.1896 - acc: 0.931 - ETA: 2:26 - loss: 0.1925 - acc: 0.928 - ETA: 2:26 - loss: 0.1944 - acc: 0.927 - ETA: 2:26 - loss: 0.1887 - acc: 0.929 - ETA: 2:25 - loss: 0.2041 - acc: 0.923 - ETA: 2:25 - loss: 0.2069 - acc: 0.922 - ETA: 2:25 - loss: 0.2157 - acc: 0.921 - ETA: 2:24 - loss: 0.2146 - acc: 0.920 - ETA: 2:24 - loss: 0.2153 - acc: 0.920 - ETA: 2:24 - loss: 0.2202 - acc: 0.916 - ETA: 2:23 - loss: 0.2149 - acc: 0.918 - ETA: 2:22 - loss: 0.2175 - acc: 0.916 - ETA: 2:22 - loss: 0.2164 - acc: 0.916 - ETA: 2:21 - loss: 0.2137 - acc: 0.9

8643/8643 [==============================] - ETA: 36s - loss: 0.1987 - acc: 0.92 - ETA: 35s - loss: 0.1988 - acc: 0.92 - ETA: 34s - loss: 0.1987 - acc: 0.92 - ETA: 34s - loss: 0.1990 - acc: 0.92 - ETA: 33s - loss: 0.1991 - acc: 0.92 - ETA: 33s - loss: 0.1991 - acc: 0.92 - ETA: 32s - loss: 0.1988 - acc: 0.92 - ETA: 32s - loss: 0.1986 - acc: 0.92 - ETA: 31s - loss: 0.1984 - acc: 0.92 - ETA: 30s - loss: 0.1984 - acc: 0.92 - ETA: 30s - loss: 0.1978 - acc: 0.92 - ETA: 29s - loss: 0.1987 - acc: 0.92 - ETA: 29s - loss: 0.1994 - acc: 0.92 - ETA: 28s - loss: 0.1996 - acc: 0.92 - ETA: 28s - loss: 0.2002 - acc: 0.92 - ETA: 27s - loss: 0.1998 - acc: 0.92 - ETA: 26s - loss: 0.2000 - acc: 0.92 - ETA: 26s - loss: 0.2002 - acc: 0.92 - ETA: 25s - loss: 0.2001 - acc: 0.92 - ETA: 25s - loss: 0.2002 - acc: 0.92 - ETA: 24s - loss: 0.2000 - acc: 0.92 - ETA: 24s - loss: 0.2005 - acc: 0.92 - ETA: 23s - loss: 0.2003 - acc: 0.92 - ETA: 22s - loss: 0.2003 - acc: 0.92 - ETA: 22s - loss: 0.2004 - acc: 0.92 - ETA: 

...

8643/8643 [==============================] - ETA: 35s - loss: 0.0874 - acc: 0.97 - ETA: 35s - loss: 0.0876 - acc: 0.96 - ETA: 34s - loss: 0.0878 - acc: 0.96 - ETA: 34s - loss: 0.0879 - acc: 0.96 - ETA: 33s - loss: 0.0875 - acc: 0.97 - ETA: 33s - loss: 0.0872 - acc: 0.97 - ETA: 32s - loss: 0.0873 - acc: 0.97 - ETA: 31s - loss: 0.0878 - acc: 0.97 - ETA: 31s - loss: 0.0880 - acc: 0.97 - ETA: 30s - loss: 0.0882 - acc: 0.97 - ETA: 30s - loss: 0.0887 - acc: 0.97 - ETA: 29s - loss: 0.0892 - acc: 0.96 - ETA: 29s - loss: 0.0891 - acc: 0.96 - ETA: 28s - loss: 0.0890 - acc: 0.96 - ETA: 27s - loss: 0.0891 - acc: 0.96 - ETA: 27s - loss: 0.0890 - acc: 0.96 - ETA: 26s - loss: 0.0889 - acc: 0.96 - ETA: 26s - loss: 0.0887 - acc: 0.96 - ETA: 25s - loss: 0.0887 - acc: 0.96 - ETA: 25s - loss: 0.0885 - acc: 0.96 - ETA: 24s - loss: 0.0886 - acc: 0.96 - ETA: 23s - loss: 0.0891 - acc: 0.96 - ETA: 23s - loss: 0.0890 - acc: 0.96 - ETA: 22s - loss: 0.0890 - acc: 0.96 - ETA: 22s - loss: 0.0891 - acc: 0.96 - ETA: 

6592/8643 [=====================>........] - ETA: 2:34 - loss: 0.0138 - acc: 1.000 - ETA: 2:32 - loss: 0.0119 - acc: 1.000 - ETA: 2:30 - loss: 0.0130 - acc: 1.000 - ETA: 2:31 - loss: 0.0397 - acc: 0.984 - ETA: 2:30 - loss: 0.0395 - acc: 0.987 - ETA: 2:30 - loss: 0.0365 - acc: 0.989 - ETA: 2:30 - loss: 0.0335 - acc: 0.991 - ETA: 2:29 - loss: 0.0313 - acc: 0.992 - ETA: 2:28 - loss: 0.0286 - acc: 0.993 - ETA: 2:28 - loss: 0.0312 - acc: 0.992 - ETA: 2:27 - loss: 0.0309 - acc: 0.993 - ETA: 2:26 - loss: 0.0293 - acc: 0.993 - ETA: 2:26 - loss: 0.0285 - acc: 0.994 - ETA: 2:25 - loss: 0.0289 - acc: 0.994 - ETA: 2:24 - loss: 0.0296 - acc: 0.993 - ETA: 2:24 - loss: 0.0285 - acc: 0.993 - ETA: 2:23 - loss: 0.0288 - acc: 0.993 - ETA: 2:22 - loss: 0.0284 - acc: 0.993 - ETA: 2:22 - loss: 0.0292 - acc: 0.992 - ETA: 2:21 - loss: 0.0280 - acc: 0.993 - ETA: 2:21 - loss: 0.0287 - acc: 0.992 - ETA: 2:20 - loss: 0.0277 - acc: 0.992 - ETA: 2:19 - loss: 0.0269 - acc: 0.993 - ETA: 2:19 - loss: 0.0298 - acc: 0.9

8643/8643 [==============================] - ETA: 35s - loss: 0.0381 - acc: 0.98 - ETA: 35s - loss: 0.0380 - acc: 0.98 - ETA: 34s - loss: 0.0384 - acc: 0.98 - ETA: 34s - loss: 0.0390 - acc: 0.98 - ETA: 33s - loss: 0.0390 - acc: 0.98 - ETA: 33s - loss: 0.0392 - acc: 0.98 - ETA: 32s - loss: 0.0390 - acc: 0.98 - ETA: 31s - loss: 0.0389 - acc: 0.98 - ETA: 31s - loss: 0.0388 - acc: 0.98 - ETA: 30s - loss: 0.0387 - acc: 0.98 - ETA: 30s - loss: 0.0388 - acc: 0.98 - ETA: 29s - loss: 0.0387 - acc: 0.98 - ETA: 29s - loss: 0.0386 - acc: 0.98 - ETA: 28s - loss: 0.0388 - acc: 0.98 - ETA: 27s - loss: 0.0389 - acc: 0.98 - ETA: 27s - loss: 0.0393 - acc: 0.98 - ETA: 26s - loss: 0.0392 - acc: 0.98 - ETA: 26s - loss: 0.0391 - acc: 0.98 - ETA: 25s - loss: 0.0391 - acc: 0.98 - ETA: 25s - loss: 0.0395 - acc: 0.98 - ETA: 24s - loss: 0.0397 - acc: 0.98 - ETA: 23s - loss: 0.0397 - acc: 0.98 - ETA: 23s - loss: 0.0401 - acc: 0.98 - ETA: 22s - loss: 0.0400 - acc: 0.98 - ETA: 22s - loss: 0.0398 - acc: 0.98 - ETA: 

6592/8643 [=====================>........] - ETA: 2:29 - loss: 0.0304 - acc: 0.979 - ETA: 2:30 - loss: 0.0276 - acc: 0.979 - ETA: 2:29 - loss: 0.0201 - acc: 0.986 - ETA: 2:30 - loss: 0.0168 - acc: 0.989 - ETA: 2:29 - loss: 0.0144 - acc: 0.991 - ETA: 2:29 - loss: 0.0133 - acc: 0.993 - ETA: 2:29 - loss: 0.0153 - acc: 0.994 - ETA: 2:28 - loss: 0.0213 - acc: 0.992 - ETA: 2:28 - loss: 0.0197 - acc: 0.993 - ETA: 2:28 - loss: 0.0186 - acc: 0.993 - ETA: 2:27 - loss: 0.0312 - acc: 0.988 - ETA: 2:26 - loss: 0.0306 - acc: 0.989 - ETA: 2:26 - loss: 0.0337 - acc: 0.987 - ETA: 2:25 - loss: 0.0331 - acc: 0.986 - ETA: 2:25 - loss: 0.0345 - acc: 0.985 - ETA: 2:24 - loss: 0.0331 - acc: 0.986 - ETA: 2:23 - loss: 0.0315 - acc: 0.987 - ETA: 2:23 - loss: 0.0300 - acc: 0.987 - ETA: 2:22 - loss: 0.0312 - acc: 0.987 - ETA: 2:22 - loss: 0.0307 - acc: 0.988 - ETA: 2:21 - loss: 0.0334 - acc: 0.987 - ETA: 2:21 - loss: 0.0322 - acc: 0.988 - ETA: 2:20 - loss: 0.0309 - acc: 0.988 - ETA: 2:19 - loss: 0.0305 - acc: 0.9

8643/8643 [==============================] - ETA: 35s - loss: 0.0203 - acc: 0.99 - ETA: 35s - loss: 0.0202 - acc: 0.99 - ETA: 34s - loss: 0.0201 - acc: 0.99 - ETA: 34s - loss: 0.0200 - acc: 0.99 - ETA: 33s - loss: 0.0200 - acc: 0.99 - ETA: 33s - loss: 0.0200 - acc: 0.99 - ETA: 32s - loss: 0.0200 - acc: 0.99 - ETA: 31s - loss: 0.0201 - acc: 0.99 - ETA: 31s - loss: 0.0201 - acc: 0.99 - ETA: 30s - loss: 0.0204 - acc: 0.99 - ETA: 30s - loss: 0.0203 - acc: 0.99 - ETA: 29s - loss: 0.0202 - acc: 0.99 - ETA: 29s - loss: 0.0202 - acc: 0.99 - ETA: 28s - loss: 0.0201 - acc: 0.99 - ETA: 27s - loss: 0.0201 - acc: 0.99 - ETA: 27s - loss: 0.0200 - acc: 0.99 - ETA: 26s - loss: 0.0199 - acc: 0.99 - ETA: 26s - loss: 0.0199 - acc: 0.99 - ETA: 25s - loss: 0.0198 - acc: 0.99 - ETA: 25s - loss: 0.0197 - acc: 0.99 - ETA: 24s - loss: 0.0198 - acc: 0.99 - ETA: 23s - loss: 0.0197 - acc: 0.99 - ETA: 23s - loss: 0.0197 - acc: 0.99 - ETA: 22s - loss: 0.0196 - acc: 0.99 - ETA: 22s - loss: 0.0196 - acc: 0.99 - ETA: 

### Accuracy for tkk trained

In [60]:
tkk_y_pred = model.predict_classes(tkk_X_test) - 1
accuracy_score(tkk_y_pred, tkk_test_labels)

0.6555407209612817

## Using pretrained fasttext vectors

In [42]:
from gensim.models import FastText

### Prepare weights for embedding layer

In [43]:
emb_model = FastText.load('emb/araneum_none_fasttextskipgram_300_5_2018.model')

In [44]:
word_vectors = emb_model.wv

In [45]:
EMBEDDING_DIM = 300
NUM_WORDS = 20000
vocabulary_size = len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

In [47]:
model = Sequential()

model.add(Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(tkk_X_train, tkk_y_train, batch_size=32, epochs=5)

Epoch 1/5


6784/8643 [======================>.......] - ETA: 7:55 - loss: 0.6819 - acc: 0.625 - ETA: 4:27 - loss: 0.6365 - acc: 0.682 - ETA: 3:23 - loss: 0.6010 - acc: 0.708 - ETA: 2:47 - loss: 0.5695 - acc: 0.729 - ETA: 2:26 - loss: 0.5645 - acc: 0.733 - ETA: 2:11 - loss: 0.5510 - acc: 0.746 - ETA: 2:02 - loss: 0.5464 - acc: 0.750 - ETA: 1:54 - loss: 0.5490 - acc: 0.750 - ETA: 1:47 - loss: 0.5529 - acc: 0.745 - ETA: 1:42 - loss: 0.5435 - acc: 0.754 - ETA: 1:37 - loss: 0.5509 - acc: 0.750 - ETA: 1:34 - loss: 0.5529 - acc: 0.750 - ETA: 1:31 - loss: 0.5610 - acc: 0.746 - ETA: 1:29 - loss: 0.5497 - acc: 0.753 - ETA: 1:27 - loss: 0.5498 - acc: 0.750 - ETA: 1:24 - loss: 0.5487 - acc: 0.750 - ETA: 1:22 - loss: 0.5462 - acc: 0.750 - ETA: 1:20 - loss: 0.5449 - acc: 0.752 - ETA: 1:19 - loss: 0.5462 - acc: 0.748 - ETA: 1:17 - loss: 0.5433 - acc: 0.750 - ETA: 1:16 - loss: 0.5410 - acc: 0.751 - ETA: 1:14 - loss: 0.5430 - acc: 0.747 - ETA: 1:13 - loss: 0.5409 - acc: 0.748 - ETA: 1:12 - loss: 0.5410 - acc: 0.7

8643/8643 [==============================] - ETA: 13s - loss: 0.4509 - acc: 0.79 - ETA: 13s - loss: 0.4507 - acc: 0.79 - ETA: 12s - loss: 0.4503 - acc: 0.79 - ETA: 12s - loss: 0.4497 - acc: 0.79 - ETA: 12s - loss: 0.4495 - acc: 0.79 - ETA: 12s - loss: 0.4499 - acc: 0.79 - ETA: 11s - loss: 0.4498 - acc: 0.79 - ETA: 11s - loss: 0.4496 - acc: 0.79 - ETA: 11s - loss: 0.4489 - acc: 0.79 - ETA: 11s - loss: 0.4489 - acc: 0.79 - ETA: 11s - loss: 0.4482 - acc: 0.79 - ETA: 10s - loss: 0.4479 - acc: 0.79 - ETA: 10s - loss: 0.4481 - acc: 0.79 - ETA: 10s - loss: 0.4479 - acc: 0.79 - ETA: 10s - loss: 0.4476 - acc: 0.79 - ETA: 9s - loss: 0.4477 - acc: 0.7993 - ETA: 9s - loss: 0.4470 - acc: 0.799 - ETA: 9s - loss: 0.4470 - acc: 0.799 - ETA: 9s - loss: 0.4460 - acc: 0.800 - ETA: 8s - loss: 0.4457 - acc: 0.800 - ETA: 8s - loss: 0.4455 - acc: 0.800 - ETA: 8s - loss: 0.4451 - acc: 0.800 - ETA: 8s - loss: 0.4447 - acc: 0.800 - ETA: 7s - loss: 0.4443 - acc: 0.800 - ETA: 7s - loss: 0.4441 - acc: 0.800 - ETA:

6848/8643 [======================>.......] - ETA: 1:00 - loss: 0.2452 - acc: 0.895 - ETA: 1:00 - loss: 0.3149 - acc: 0.864 - ETA: 1:01 - loss: 0.2975 - acc: 0.875 - ETA: 1:01 - loss: 0.2840 - acc: 0.880 - ETA: 1:00 - loss: 0.2774 - acc: 0.881 - ETA: 1:00 - loss: 0.2713 - acc: 0.887 - ETA: 1:01 - loss: 0.2710 - acc: 0.883 - ETA: 1:01 - loss: 0.2741 - acc: 0.880 - ETA: 1:00 - loss: 0.2767 - acc: 0.877 - ETA: 1:00 - loss: 0.2746 - acc: 0.882 - ETA: 1:00 - loss: 0.2699 - acc: 0.881 - ETA: 59s - loss: 0.2671 - acc: 0.883 - ETA: 59s - loss: 0.2703 - acc: 0.87 - ETA: 59s - loss: 0.2721 - acc: 0.88 - ETA: 58s - loss: 0.2642 - acc: 0.88 - ETA: 58s - loss: 0.2600 - acc: 0.88 - ETA: 58s - loss: 0.2675 - acc: 0.88 - ETA: 58s - loss: 0.2741 - acc: 0.88 - ETA: 58s - loss: 0.2766 - acc: 0.88 - ETA: 58s - loss: 0.2775 - acc: 0.88 - ETA: 58s - loss: 0.2744 - acc: 0.88 - ETA: 57s - loss: 0.2747 - acc: 0.88 - ETA: 57s - loss: 0.2763 - acc: 0.88 - ETA: 56s - loss: 0.2828 - acc: 0.88 - ETA: 56s - loss: 0.2

8643/8643 [==============================] - ETA: 12s - loss: 0.2696 - acc: 0.88 - ETA: 12s - loss: 0.2694 - acc: 0.88 - ETA: 12s - loss: 0.2693 - acc: 0.88 - ETA: 12s - loss: 0.2698 - acc: 0.88 - ETA: 11s - loss: 0.2696 - acc: 0.88 - ETA: 11s - loss: 0.2694 - acc: 0.88 - ETA: 11s - loss: 0.2694 - acc: 0.88 - ETA: 11s - loss: 0.2694 - acc: 0.88 - ETA: 10s - loss: 0.2692 - acc: 0.88 - ETA: 10s - loss: 0.2693 - acc: 0.88 - ETA: 10s - loss: 0.2696 - acc: 0.88 - ETA: 10s - loss: 0.2710 - acc: 0.88 - ETA: 9s - loss: 0.2712 - acc: 0.8876 - ETA: 9s - loss: 0.2713 - acc: 0.887 - ETA: 9s - loss: 0.2713 - acc: 0.887 - ETA: 9s - loss: 0.2712 - acc: 0.887 - ETA: 9s - loss: 0.2711 - acc: 0.887 - ETA: 8s - loss: 0.2713 - acc: 0.887 - ETA: 8s - loss: 0.2709 - acc: 0.887 - ETA: 8s - loss: 0.2706 - acc: 0.887 - ETA: 8s - loss: 0.2702 - acc: 0.887 - ETA: 7s - loss: 0.2702 - acc: 0.887 - ETA: 7s - loss: 0.2702 - acc: 0.887 - ETA: 7s - loss: 0.2700 - acc: 0.888 - ETA: 7s - loss: 0.2697 - acc: 0.888 - ETA:

...

8643/8643 [==============================] - ETA: 12s - loss: 0.1641 - acc: 0.93 - ETA: 12s - loss: 0.1642 - acc: 0.93 - ETA: 12s - loss: 0.1652 - acc: 0.93 - ETA: 12s - loss: 0.1652 - acc: 0.93 - ETA: 12s - loss: 0.1653 - acc: 0.93 - ETA: 11s - loss: 0.1653 - acc: 0.93 - ETA: 11s - loss: 0.1657 - acc: 0.93 - ETA: 11s - loss: 0.1658 - acc: 0.93 - ETA: 11s - loss: 0.1654 - acc: 0.93 - ETA: 10s - loss: 0.1653 - acc: 0.93 - ETA: 10s - loss: 0.1653 - acc: 0.93 - ETA: 10s - loss: 0.1656 - acc: 0.93 - ETA: 10s - loss: 0.1659 - acc: 0.93 - ETA: 9s - loss: 0.1657 - acc: 0.9355 - ETA: 9s - loss: 0.1654 - acc: 0.935 - ETA: 9s - loss: 0.1654 - acc: 0.935 - ETA: 9s - loss: 0.1653 - acc: 0.935 - ETA: 8s - loss: 0.1650 - acc: 0.935 - ETA: 8s - loss: 0.1654 - acc: 0.935 - ETA: 8s - loss: 0.1657 - acc: 0.935 - ETA: 8s - loss: 0.1655 - acc: 0.935 - ETA: 7s - loss: 0.1653 - acc: 0.935 - ETA: 7s - loss: 0.1651 - acc: 0.935 - ETA: 7s - loss: 0.1651 - acc: 0.935 - ETA: 7s - loss: 0.1649 - acc: 0.935 - ETA:

6848/8643 [======================>.......] - ETA: 1:04 - loss: 0.0398 - acc: 1.000 - ETA: 1:02 - loss: 0.0711 - acc: 0.984 - ETA: 1:02 - loss: 0.0692 - acc: 0.986 - ETA: 1:01 - loss: 0.0616 - acc: 0.989 - ETA: 1:01 - loss: 0.0677 - acc: 0.983 - ETA: 1:01 - loss: 0.0730 - acc: 0.977 - ETA: 1:00 - loss: 0.0792 - acc: 0.971 - ETA: 1:00 - loss: 0.0843 - acc: 0.966 - ETA: 1:00 - loss: 0.0786 - acc: 0.969 - ETA: 59s - loss: 0.0759 - acc: 0.971 - ETA: 59s - loss: 0.0757 - acc: 0.97 - ETA: 59s - loss: 0.0778 - acc: 0.97 - ETA: 58s - loss: 0.0801 - acc: 0.97 - ETA: 58s - loss: 0.0860 - acc: 0.96 - ETA: 57s - loss: 0.0821 - acc: 0.97 - ETA: 57s - loss: 0.0808 - acc: 0.97 - ETA: 57s - loss: 0.0868 - acc: 0.96 - ETA: 57s - loss: 0.0839 - acc: 0.97 - ETA: 57s - loss: 0.0835 - acc: 0.97 - ETA: 56s - loss: 0.0845 - acc: 0.97 - ETA: 56s - loss: 0.0820 - acc: 0.97 - ETA: 56s - loss: 0.0849 - acc: 0.97 - ETA: 56s - loss: 0.0872 - acc: 0.97 - ETA: 56s - loss: 0.0861 - acc: 0.97 - ETA: 56s - loss: 0.0855 

8643/8643 [==============================] - ETA: 12s - loss: 0.0987 - acc: 0.96 - ETA: 12s - loss: 0.0988 - acc: 0.96 - ETA: 12s - loss: 0.0987 - acc: 0.96 - ETA: 12s - loss: 0.0985 - acc: 0.96 - ETA: 11s - loss: 0.0987 - acc: 0.96 - ETA: 11s - loss: 0.0989 - acc: 0.96 - ETA: 11s - loss: 0.0987 - acc: 0.96 - ETA: 11s - loss: 0.0990 - acc: 0.96 - ETA: 10s - loss: 0.0991 - acc: 0.96 - ETA: 10s - loss: 0.0987 - acc: 0.96 - ETA: 10s - loss: 0.0985 - acc: 0.96 - ETA: 10s - loss: 0.0984 - acc: 0.96 - ETA: 10s - loss: 0.0984 - acc: 0.96 - ETA: 9s - loss: 0.0985 - acc: 0.9648 - ETA: 9s - loss: 0.0982 - acc: 0.964 - ETA: 9s - loss: 0.0983 - acc: 0.964 - ETA: 9s - loss: 0.0988 - acc: 0.964 - ETA: 8s - loss: 0.0989 - acc: 0.964 - ETA: 8s - loss: 0.0989 - acc: 0.964 - ETA: 8s - loss: 0.0988 - acc: 0.964 - ETA: 8s - loss: 0.0987 - acc: 0.964 - ETA: 7s - loss: 0.0989 - acc: 0.964 - ETA: 7s - loss: 0.0987 - acc: 0.964 - ETA: 7s - loss: 0.0986 - acc: 0.964 - ETA: 7s - loss: 0.0987 - acc: 0.964 - ETA:

6880/8643 [======================>.......] - ETA: 58s - loss: 0.0730 - acc: 0.97 - ETA: 57s - loss: 0.0833 - acc: 0.96 - ETA: 57s - loss: 0.0655 - acc: 0.97 - ETA: 58s - loss: 0.0620 - acc: 0.97 - ETA: 58s - loss: 0.0608 - acc: 0.98 - ETA: 59s - loss: 0.0654 - acc: 0.98 - ETA: 59s - loss: 0.0590 - acc: 0.98 - ETA: 59s - loss: 0.0555 - acc: 0.98 - ETA: 58s - loss: 0.0576 - acc: 0.98 - ETA: 59s - loss: 0.0601 - acc: 0.98 - ETA: 59s - loss: 0.0583 - acc: 0.98 - ETA: 1:00 - loss: 0.0554 - acc: 0.987 - ETA: 1:00 - loss: 0.0535 - acc: 0.987 - ETA: 1:00 - loss: 0.0535 - acc: 0.986 - ETA: 1:00 - loss: 0.0539 - acc: 0.986 - ETA: 59s - loss: 0.0528 - acc: 0.987 - ETA: 59s - loss: 0.0522 - acc: 0.98 - ETA: 59s - loss: 0.0509 - acc: 0.98 - ETA: 58s - loss: 0.0509 - acc: 0.98 - ETA: 58s - loss: 0.0507 - acc: 0.98 - ETA: 58s - loss: 0.0509 - acc: 0.98 - ETA: 57s - loss: 0.0524 - acc: 0.98 - ETA: 57s - loss: 0.0568 - acc: 0.98 - ETA: 57s - loss: 0.0574 - acc: 0.98 - ETA: 56s - loss: 0.0565 - acc: 0.9

8643/8643 [==============================] - ETA: 13s - loss: 0.0616 - acc: 0.97 - ETA: 13s - loss: 0.0614 - acc: 0.98 - ETA: 13s - loss: 0.0611 - acc: 0.98 - ETA: 13s - loss: 0.0623 - acc: 0.97 - ETA: 12s - loss: 0.0622 - acc: 0.97 - ETA: 12s - loss: 0.0623 - acc: 0.97 - ETA: 12s - loss: 0.0622 - acc: 0.97 - ETA: 12s - loss: 0.0621 - acc: 0.97 - ETA: 11s - loss: 0.0621 - acc: 0.97 - ETA: 11s - loss: 0.0621 - acc: 0.97 - ETA: 11s - loss: 0.0621 - acc: 0.97 - ETA: 11s - loss: 0.0620 - acc: 0.97 - ETA: 10s - loss: 0.0618 - acc: 0.97 - ETA: 10s - loss: 0.0618 - acc: 0.97 - ETA: 10s - loss: 0.0617 - acc: 0.97 - ETA: 10s - loss: 0.0615 - acc: 0.97 - ETA: 9s - loss: 0.0614 - acc: 0.9800 - ETA: 9s - loss: 0.0613 - acc: 0.980 - ETA: 9s - loss: 0.0612 - acc: 0.980 - ETA: 8s - loss: 0.0610 - acc: 0.980 - ETA: 8s - loss: 0.0611 - acc: 0.980 - ETA: 8s - loss: 0.0611 - acc: 0.980 - ETA: 8s - loss: 0.0609 - acc: 0.980 - ETA: 7s - loss: 0.0609 - acc: 0.980 - ETA: 7s - loss: 0.0608 - acc: 0.980 - ETA:

### Accuracy for tkk pretrained

In [48]:
tkk_predictions = model.predict_classes(tkk_X_test) - 1
accuracy_score(tkk_predictions, tkk_test_labels)

0.6644414775255897

# Bank

## Without pretrained

### Data prepration

In [54]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(bank_train_text)

bank_X_train = pad_sequences(tokenizer.texts_to_sequences(bank_train_text), maxlen=100)
bank_X_test = pad_sequences(tokenizer.texts_to_sequences(bank_test_text), maxlen=100)

bank_word_index = tokenizer.word_index

encoder = LabelBinarizer()
encoder.fit(bank_train_labels)
bank_y_train = encoder.transform(bank_train_labels)
bank_y_test = encoder.transform(bank_test_labels)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

### Train model

In [52]:
model = Sequential()

model.add(Embedding(10000, 1000, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])

model.fit(bank_X_train, bank_y_train, batch_size=32, epochs=5)

Epoch 1/5


6528/9392 [===================>..........] - ETA: 13:30 - loss: 0.6939 - acc: 0.34 - ETA: 8:24 - loss: 0.6234 - acc: 0.6198 - ETA: 6:47 - loss: 0.5627 - acc: 0.711 - ETA: 5:54 - loss: 0.5559 - acc: 0.721 - ETA: 5:17 - loss: 0.5175 - acc: 0.752 - ETA: 4:54 - loss: 0.4943 - acc: 0.769 - ETA: 4:36 - loss: 0.4765 - acc: 0.781 - ETA: 4:21 - loss: 0.4643 - acc: 0.790 - ETA: 4:12 - loss: 0.4769 - acc: 0.785 - ETA: 4:09 - loss: 0.4928 - acc: 0.778 - ETA: 4:02 - loss: 0.4921 - acc: 0.781 - ETA: 3:56 - loss: 0.4858 - acc: 0.783 - ETA: 3:49 - loss: 0.4797 - acc: 0.789 - ETA: 3:45 - loss: 0.4838 - acc: 0.783 - ETA: 3:41 - loss: 0.4754 - acc: 0.791 - ETA: 3:36 - loss: 0.4694 - acc: 0.794 - ETA: 3:33 - loss: 0.4720 - acc: 0.789 - ETA: 3:29 - loss: 0.4682 - acc: 0.792 - ETA: 3:26 - loss: 0.4626 - acc: 0.796 - ETA: 3:24 - loss: 0.4542 - acc: 0.801 - ETA: 3:23 - loss: 0.4525 - acc: 0.802 - ETA: 3:20 - loss: 0.4492 - acc: 0.803 - ETA: 3:17 - loss: 0.4464 - acc: 0.803 - ETA: 3:15 - loss: 0.4395 - acc: 0.

9392/9392 [==============================] - ETA: 53s - loss: 0.2724 - acc: 0.88 - ETA: 52s - loss: 0.2727 - acc: 0.88 - ETA: 52s - loss: 0.2720 - acc: 0.88 - ETA: 51s - loss: 0.2724 - acc: 0.88 - ETA: 50s - loss: 0.2725 - acc: 0.88 - ETA: 50s - loss: 0.2721 - acc: 0.88 - ETA: 49s - loss: 0.2730 - acc: 0.88 - ETA: 48s - loss: 0.2734 - acc: 0.88 - ETA: 48s - loss: 0.2732 - acc: 0.88 - ETA: 47s - loss: 0.2728 - acc: 0.88 - ETA: 47s - loss: 0.2725 - acc: 0.88 - ETA: 46s - loss: 0.2725 - acc: 0.88 - ETA: 45s - loss: 0.2726 - acc: 0.88 - ETA: 45s - loss: 0.2724 - acc: 0.88 - ETA: 44s - loss: 0.2724 - acc: 0.88 - ETA: 44s - loss: 0.2721 - acc: 0.88 - ETA: 43s - loss: 0.2721 - acc: 0.88 - ETA: 42s - loss: 0.2715 - acc: 0.88 - ETA: 42s - loss: 0.2713 - acc: 0.88 - ETA: 41s - loss: 0.2708 - acc: 0.88 - ETA: 41s - loss: 0.2705 - acc: 0.88 - ETA: 40s - loss: 0.2703 - acc: 0.88 - ETA: 39s - loss: 0.2698 - acc: 0.88 - ETA: 39s - loss: 0.2701 - acc: 0.88 - ETA: 38s - loss: 0.2697 - acc: 0.88 - ETA: 

6560/9392 [===================>..........] - ETA: 3:06 - loss: 0.1883 - acc: 0.958 - ETA: 3:15 - loss: 0.1231 - acc: 0.979 - ETA: 3:17 - loss: 0.1157 - acc: 0.979 - ETA: 3:11 - loss: 0.1021 - acc: 0.981 - ETA: 3:05 - loss: 0.1065 - acc: 0.979 - ETA: 3:01 - loss: 0.1039 - acc: 0.975 - ETA: 2:59 - loss: 0.0982 - acc: 0.979 - ETA: 2:58 - loss: 0.1021 - acc: 0.976 - ETA: 2:57 - loss: 0.0973 - acc: 0.979 - ETA: 2:56 - loss: 0.1070 - acc: 0.976 - ETA: 2:55 - loss: 0.1029 - acc: 0.975 - ETA: 2:55 - loss: 0.0982 - acc: 0.976 - ETA: 2:54 - loss: 0.1038 - acc: 0.969 - ETA: 2:53 - loss: 0.1122 - acc: 0.968 - ETA: 2:51 - loss: 0.1116 - acc: 0.968 - ETA: 2:50 - loss: 0.1084 - acc: 0.968 - ETA: 2:50 - loss: 0.1057 - acc: 0.970 - ETA: 2:48 - loss: 0.1083 - acc: 0.969 - ETA: 2:47 - loss: 0.1145 - acc: 0.965 - ETA: 2:46 - loss: 0.1134 - acc: 0.965 - ETA: 2:45 - loss: 0.1127 - acc: 0.965 - ETA: 2:44 - loss: 0.1157 - acc: 0.964 - ETA: 2:43 - loss: 0.1132 - acc: 0.965 - ETA: 2:42 - loss: 0.1117 - acc: 0.9

9392/9392 [==============================] - ETA: 51s - loss: 0.1184 - acc: 0.95 - ETA: 51s - loss: 0.1180 - acc: 0.95 - ETA: 50s - loss: 0.1183 - acc: 0.95 - ETA: 49s - loss: 0.1184 - acc: 0.95 - ETA: 49s - loss: 0.1183 - acc: 0.95 - ETA: 48s - loss: 0.1180 - acc: 0.95 - ETA: 48s - loss: 0.1181 - acc: 0.95 - ETA: 47s - loss: 0.1182 - acc: 0.95 - ETA: 47s - loss: 0.1185 - acc: 0.95 - ETA: 46s - loss: 0.1195 - acc: 0.95 - ETA: 45s - loss: 0.1197 - acc: 0.95 - ETA: 45s - loss: 0.1195 - acc: 0.95 - ETA: 44s - loss: 0.1196 - acc: 0.95 - ETA: 44s - loss: 0.1202 - acc: 0.95 - ETA: 43s - loss: 0.1202 - acc: 0.95 - ETA: 42s - loss: 0.1202 - acc: 0.95 - ETA: 42s - loss: 0.1203 - acc: 0.95 - ETA: 41s - loss: 0.1204 - acc: 0.95 - ETA: 41s - loss: 0.1208 - acc: 0.95 - ETA: 40s - loss: 0.1211 - acc: 0.95 - ETA: 39s - loss: 0.1209 - acc: 0.95 - ETA: 39s - loss: 0.1209 - acc: 0.95 - ETA: 38s - loss: 0.1209 - acc: 0.95 - ETA: 38s - loss: 0.1207 - acc: 0.95 - ETA: 37s - loss: 0.1207 - acc: 0.95 - ETA: 

...

9392/9392 [==============================] - ETA: 51s - loss: 0.0471 - acc: 0.98 - ETA: 50s - loss: 0.0469 - acc: 0.98 - ETA: 49s - loss: 0.0472 - acc: 0.98 - ETA: 49s - loss: 0.0470 - acc: 0.98 - ETA: 48s - loss: 0.0468 - acc: 0.98 - ETA: 48s - loss: 0.0470 - acc: 0.98 - ETA: 47s - loss: 0.0469 - acc: 0.98 - ETA: 46s - loss: 0.0467 - acc: 0.98 - ETA: 46s - loss: 0.0467 - acc: 0.98 - ETA: 45s - loss: 0.0465 - acc: 0.98 - ETA: 45s - loss: 0.0464 - acc: 0.98 - ETA: 44s - loss: 0.0465 - acc: 0.98 - ETA: 44s - loss: 0.0469 - acc: 0.98 - ETA: 43s - loss: 0.0468 - acc: 0.98 - ETA: 42s - loss: 0.0471 - acc: 0.98 - ETA: 42s - loss: 0.0469 - acc: 0.98 - ETA: 41s - loss: 0.0471 - acc: 0.98 - ETA: 41s - loss: 0.0470 - acc: 0.98 - ETA: 40s - loss: 0.0470 - acc: 0.98 - ETA: 39s - loss: 0.0469 - acc: 0.98 - ETA: 39s - loss: 0.0469 - acc: 0.98 - ETA: 38s - loss: 0.0472 - acc: 0.98 - ETA: 38s - loss: 0.0475 - acc: 0.98 - ETA: 37s - loss: 0.0480 - acc: 0.98 - ETA: 37s - loss: 0.0478 - acc: 0.98 - ETA: 

6560/9392 [===================>..........] - ETA: 2:43 - loss: 0.0320 - acc: 0.979 - ETA: 2:45 - loss: 0.0184 - acc: 0.989 - ETA: 2:44 - loss: 0.0138 - acc: 0.993 - ETA: 2:44 - loss: 0.0137 - acc: 0.994 - ETA: 2:45 - loss: 0.0130 - acc: 0.995 - ETA: 2:44 - loss: 0.0134 - acc: 0.996 - ETA: 2:44 - loss: 0.0152 - acc: 0.995 - ETA: 2:43 - loss: 0.0147 - acc: 0.996 - ETA: 2:43 - loss: 0.0143 - acc: 0.996 - ETA: 2:42 - loss: 0.0136 - acc: 0.996 - ETA: 2:41 - loss: 0.0134 - acc: 0.997 - ETA: 2:41 - loss: 0.0175 - acc: 0.994 - ETA: 2:40 - loss: 0.0171 - acc: 0.995 - ETA: 2:39 - loss: 0.0163 - acc: 0.995 - ETA: 2:38 - loss: 0.0162 - acc: 0.995 - ETA: 2:38 - loss: 0.0179 - acc: 0.994 - ETA: 2:37 - loss: 0.0178 - acc: 0.995 - ETA: 2:37 - loss: 0.0178 - acc: 0.995 - ETA: 2:36 - loss: 0.0202 - acc: 0.994 - ETA: 2:35 - loss: 0.0194 - acc: 0.994 - ETA: 2:35 - loss: 0.0188 - acc: 0.995 - ETA: 2:35 - loss: 0.0185 - acc: 0.995 - ETA: 2:34 - loss: 0.0178 - acc: 0.995 - ETA: 2:33 - loss: 0.0175 - acc: 0.9

9392/9392 [==============================] - ETA: 51s - loss: 0.0237 - acc: 0.99 - ETA: 51s - loss: 0.0236 - acc: 0.99 - ETA: 50s - loss: 0.0236 - acc: 0.99 - ETA: 49s - loss: 0.0235 - acc: 0.99 - ETA: 49s - loss: 0.0234 - acc: 0.99 - ETA: 48s - loss: 0.0233 - acc: 0.99 - ETA: 48s - loss: 0.0232 - acc: 0.99 - ETA: 47s - loss: 0.0232 - acc: 0.99 - ETA: 46s - loss: 0.0231 - acc: 0.99 - ETA: 46s - loss: 0.0230 - acc: 0.99 - ETA: 45s - loss: 0.0230 - acc: 0.99 - ETA: 45s - loss: 0.0229 - acc: 0.99 - ETA: 44s - loss: 0.0229 - acc: 0.99 - ETA: 43s - loss: 0.0228 - acc: 0.99 - ETA: 43s - loss: 0.0228 - acc: 0.99 - ETA: 42s - loss: 0.0227 - acc: 0.99 - ETA: 42s - loss: 0.0226 - acc: 0.99 - ETA: 41s - loss: 0.0225 - acc: 0.99 - ETA: 41s - loss: 0.0224 - acc: 0.99 - ETA: 40s - loss: 0.0224 - acc: 0.99 - ETA: 39s - loss: 0.0225 - acc: 0.99 - ETA: 39s - loss: 0.0224 - acc: 0.99 - ETA: 38s - loss: 0.0225 - acc: 0.99 - ETA: 38s - loss: 0.0224 - acc: 0.99 - ETA: 37s - loss: 0.0224 - acc: 0.99 - ETA: 

6560/9392 [===================>..........] - ETA: 2:55 - loss: 0.0030 - acc: 1.000 - ETA: 2:54 - loss: 0.0040 - acc: 1.000 - ETA: 2:54 - loss: 0.0039 - acc: 1.000 - ETA: 2:52 - loss: 0.0035 - acc: 1.000 - ETA: 2:51 - loss: 0.0038 - acc: 1.000 - ETA: 2:51 - loss: 0.0038 - acc: 1.000 - ETA: 2:51 - loss: 0.0039 - acc: 1.000 - ETA: 2:50 - loss: 0.0047 - acc: 1.000 - ETA: 2:49 - loss: 0.0044 - acc: 1.000 - ETA: 2:48 - loss: 0.0084 - acc: 0.997 - ETA: 2:48 - loss: 0.0133 - acc: 0.996 - ETA: 2:47 - loss: 0.0131 - acc: 0.996 - ETA: 2:46 - loss: 0.0123 - acc: 0.996 - ETA: 2:45 - loss: 0.0116 - acc: 0.997 - ETA: 2:44 - loss: 0.0111 - acc: 0.997 - ETA: 2:44 - loss: 0.0110 - acc: 0.997 - ETA: 2:43 - loss: 0.0104 - acc: 0.997 - ETA: 2:43 - loss: 0.0099 - acc: 0.997 - ETA: 2:43 - loss: 0.0096 - acc: 0.997 - ETA: 2:42 - loss: 0.0092 - acc: 0.997 - ETA: 2:41 - loss: 0.0090 - acc: 0.998 - ETA: 2:40 - loss: 0.0088 - acc: 0.998 - ETA: 2:40 - loss: 0.0085 - acc: 0.998 - ETA: 2:39 - loss: 0.0082 - acc: 0.9

9392/9392 [==============================] - ETA: 50s - loss: 0.0115 - acc: 0.99 - ETA: 49s - loss: 0.0114 - acc: 0.99 - ETA: 49s - loss: 0.0115 - acc: 0.99 - ETA: 48s - loss: 0.0115 - acc: 0.99 - ETA: 48s - loss: 0.0115 - acc: 0.99 - ETA: 47s - loss: 0.0115 - acc: 0.99 - ETA: 46s - loss: 0.0114 - acc: 0.99 - ETA: 46s - loss: 0.0114 - acc: 0.99 - ETA: 45s - loss: 0.0113 - acc: 0.99 - ETA: 45s - loss: 0.0113 - acc: 0.99 - ETA: 44s - loss: 0.0113 - acc: 0.99 - ETA: 44s - loss: 0.0112 - acc: 0.99 - ETA: 43s - loss: 0.0112 - acc: 0.99 - ETA: 42s - loss: 0.0111 - acc: 0.99 - ETA: 42s - loss: 0.0111 - acc: 0.99 - ETA: 41s - loss: 0.0111 - acc: 0.99 - ETA: 41s - loss: 0.0110 - acc: 0.99 - ETA: 40s - loss: 0.0110 - acc: 0.99 - ETA: 40s - loss: 0.0110 - acc: 0.99 - ETA: 39s - loss: 0.0109 - acc: 0.99 - ETA: 38s - loss: 0.0109 - acc: 0.99 - ETA: 38s - loss: 0.0109 - acc: 0.99 - ETA: 37s - loss: 0.0108 - acc: 0.99 - ETA: 37s - loss: 0.0108 - acc: 0.99 - ETA: 36s - loss: 0.0108 - acc: 0.99 - ETA: 

### Accuracy for bank trained

In [53]:
bank_predictions = model.predict_classes(bank_X_test) - 1
accuracy_score(bank_predictions, bank_test_labels)

0.7120434651373377

## Using pretrained

In [55]:
EMBEDDING_DIM = 300
NUM_WORDS = 20000
vocabulary_size = len(bank_word_index) + 1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in bank_word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

In [57]:
model = Sequential()

model.add(Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(bank_X_train, bank_y_train, batch_size=64, epochs=5)

Epoch 1/5
9392/9392 [==============================] - ETA: 4:27 - loss: 0.7586 - acc: 0.171 - ETA: 2:37 - loss: 0.6338 - acc: 0.526 - ETA: 2:00 - loss: 0.5796 - acc: 0.621 - ETA: 1:42 - loss: 0.5432 - acc: 0.677 - ETA: 1:31 - loss: 0.5309 - acc: 0.704 - ETA: 1:23 - loss: 0.5212 - acc: 0.720 - ETA: 1:18 - loss: 0.5072 - acc: 0.735 - ETA: 1:13 - loss: 0.4899 - acc: 0.751 - ETA: 1:10 - loss: 0.4742 - acc: 0.763 - ETA: 1:07 - loss: 0.4627 - acc: 0.774 - ETA: 1:05 - loss: 0.4630 - acc: 0.775 - ETA: 1:03 - loss: 0.4535 - acc: 0.781 - ETA: 1:01 - loss: 0.4503 - acc: 0.784 - ETA: 1:00 - loss: 0.4444 - acc: 0.790 - ETA: 59s - loss: 0.4367 - acc: 0.795 - ETA: 58s - loss: 0.4357 - acc: 0.79 - ETA: 56s - loss: 0.4300 - acc: 0.80 - ETA: 55s - loss: 0.4233 - acc: 0.80 - ETA: 54s - loss: 0.4160 - acc: 0.80 - ETA: 53s - loss: 0.4159 - acc: 0.81 - ETA: 53s - loss: 0.4125 - acc: 0.81 - ETA: 52s - loss: 0.4088 - acc: 0.81 - ETA: 51s - loss: 0.4045 - acc: 0.81 - ETA: 50s - loss: 0.4040 - acc: 0.81 - ETA:

9392/9392 [==============================] - ETA: 51s - loss: 0.1982 - acc: 0.91 - ETA: 51s - loss: 0.2157 - acc: 0.90 - ETA: 51s - loss: 0.1915 - acc: 0.91 - ETA: 50s - loss: 0.1828 - acc: 0.91 - ETA: 50s - loss: 0.1957 - acc: 0.91 - ETA: 50s - loss: 0.1908 - acc: 0.92 - ETA: 50s - loss: 0.1876 - acc: 0.92 - ETA: 50s - loss: 0.1856 - acc: 0.92 - ETA: 49s - loss: 0.1917 - acc: 0.91 - ETA: 49s - loss: 0.1904 - acc: 0.92 - ETA: 48s - loss: 0.1882 - acc: 0.92 - ETA: 48s - loss: 0.1879 - acc: 0.92 - ETA: 48s - loss: 0.1852 - acc: 0.92 - ETA: 48s - loss: 0.1860 - acc: 0.92 - ETA: 47s - loss: 0.1851 - acc: 0.92 - ETA: 47s - loss: 0.1847 - acc: 0.92 - ETA: 47s - loss: 0.1859 - acc: 0.92 - ETA: 46s - loss: 0.1818 - acc: 0.92 - ETA: 46s - loss: 0.1832 - acc: 0.92 - ETA: 45s - loss: 0.1846 - acc: 0.92 - ETA: 45s - loss: 0.1851 - acc: 0.92 - ETA: 45s - loss: 0.1894 - acc: 0.92 - ETA: 44s - loss: 0.1894 - acc: 0.92 - ETA: 44s - loss: 0.1920 - acc: 0.92 - ETA: 44s - loss: 0.1937 - acc: 0.92 - ETA: 

9392/9392 [==============================] - ETA: 50s - loss: 0.1432 - acc: 0.94 - ETA: 50s - loss: 0.1502 - acc: 0.94 - ETA: 52s - loss: 0.1334 - acc: 0.95 - ETA: 51s - loss: 0.1441 - acc: 0.94 - ETA: 50s - loss: 0.1320 - acc: 0.95 - ETA: 50s - loss: 0.1322 - acc: 0.95 - ETA: 49s - loss: 0.1252 - acc: 0.95 - ETA: 49s - loss: 0.1270 - acc: 0.95 - ETA: 48s - loss: 0.1271 - acc: 0.95 - ETA: 48s - loss: 0.1320 - acc: 0.95 - ETA: 48s - loss: 0.1312 - acc: 0.95 - ETA: 48s - loss: 0.1282 - acc: 0.95 - ETA: 47s - loss: 0.1243 - acc: 0.95 - ETA: 47s - loss: 0.1230 - acc: 0.95 - ETA: 46s - loss: 0.1217 - acc: 0.95 - ETA: 46s - loss: 0.1260 - acc: 0.95 - ETA: 46s - loss: 0.1222 - acc: 0.95 - ETA: 45s - loss: 0.1236 - acc: 0.95 - ETA: 45s - loss: 0.1286 - acc: 0.95 - ETA: 45s - loss: 0.1258 - acc: 0.95 - ETA: 44s - loss: 0.1250 - acc: 0.95 - ETA: 44s - loss: 0.1272 - acc: 0.95 - ETA: 44s - loss: 0.1249 - acc: 0.95 - ETA: 43s - loss: 0.1236 - acc: 0.95 - ETA: 43s - loss: 0.1216 - acc: 0.95 - ETA: 

9392/9392 [==============================] - ETA: 49s - loss: 0.0949 - acc: 0.96 - ETA: 50s - loss: 0.0821 - acc: 0.97 - ETA: 49s - loss: 0.0674 - acc: 0.98 - ETA: 50s - loss: 0.0619 - acc: 0.98 - ETA: 49s - loss: 0.0617 - acc: 0.98 - ETA: 49s - loss: 0.0611 - acc: 0.98 - ETA: 49s - loss: 0.0639 - acc: 0.98 - ETA: 49s - loss: 0.0603 - acc: 0.98 - ETA: 49s - loss: 0.0590 - acc: 0.98 - ETA: 48s - loss: 0.0582 - acc: 0.98 - ETA: 48s - loss: 0.0549 - acc: 0.98 - ETA: 47s - loss: 0.0550 - acc: 0.98 - ETA: 47s - loss: 0.0545 - acc: 0.98 - ETA: 47s - loss: 0.0548 - acc: 0.98 - ETA: 47s - loss: 0.0534 - acc: 0.98 - ETA: 46s - loss: 0.0538 - acc: 0.98 - ETA: 46s - loss: 0.0563 - acc: 0.98 - ETA: 46s - loss: 0.0567 - acc: 0.98 - ETA: 45s - loss: 0.0568 - acc: 0.98 - ETA: 45s - loss: 0.0563 - acc: 0.98 - ETA: 44s - loss: 0.0577 - acc: 0.98 - ETA: 44s - loss: 0.0563 - acc: 0.98 - ETA: 44s - loss: 0.0557 - acc: 0.98 - ETA: 43s - loss: 0.0573 - acc: 0.98 - ETA: 43s - loss: 0.0572 - acc: 0.98 - ETA: 

9392/9392 [==============================] - ETA: 48s - loss: 0.0143 - acc: 1.00 - ETA: 49s - loss: 0.0243 - acc: 0.99 - ETA: 49s - loss: 0.0257 - acc: 0.99 - ETA: 49s - loss: 0.0342 - acc: 0.98 - ETA: 49s - loss: 0.0343 - acc: 0.98 - ETA: 49s - loss: 0.0309 - acc: 0.99 - ETA: 49s - loss: 0.0395 - acc: 0.98 - ETA: 48s - loss: 0.0394 - acc: 0.98 - ETA: 48s - loss: 0.0368 - acc: 0.98 - ETA: 48s - loss: 0.0346 - acc: 0.99 - ETA: 47s - loss: 0.0366 - acc: 0.98 - ETA: 47s - loss: 0.0359 - acc: 0.98 - ETA: 47s - loss: 0.0347 - acc: 0.98 - ETA: 46s - loss: 0.0371 - acc: 0.98 - ETA: 46s - loss: 0.0396 - acc: 0.98 - ETA: 46s - loss: 0.0384 - acc: 0.98 - ETA: 46s - loss: 0.0373 - acc: 0.98 - ETA: 45s - loss: 0.0368 - acc: 0.98 - ETA: 45s - loss: 0.0357 - acc: 0.98 - ETA: 45s - loss: 0.0357 - acc: 0.98 - ETA: 44s - loss: 0.0349 - acc: 0.98 - ETA: 44s - loss: 0.0376 - acc: 0.98 - ETA: 44s - loss: 0.0370 - acc: 0.98 - ETA: 43s - loss: 0.0365 - acc: 0.98 - ETA: 43s - loss: 0.0372 - acc: 0.98 - ETA: 

### Accuracy for bank pretrained

In [58]:
bank_predictions = model.predict_classes(bank_X_test) - 1
accuracy_score(bank_predictions, bank_test_labels)

0.6884998490793842